In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 49.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 88.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch

print("PyTorch version:", torch.__version__)


PyTorch version: 2.1.0+cu118


In [3]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import time

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cuda


In [7]:
# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation and test
val_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_val_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=val_test_transform)

# Split train_val_dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# Apply val_test_transform to the validation set
val_dataset.dataset.transform = val_test_transform

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

100.0%


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 100)
# Move models to device
teacher = teacher.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
24.0%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.

In [9]:

model_path = 'best_teacher_model.pth'
# Load the model weights

teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [128]:
teacher_accuracy = evaluate(teacher, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")

Teacher Model Test Accuracy: 80.89%


In [129]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 100)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
28.1%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100.0%


In [27]:

# model_path = 'student_before_pruning.pth'
# # Load the model weights
# student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

In [130]:
# Logits normalization function
def normalize(logit):
    mean = logit.mean(dim=-1, keepdim=True)
    stdv = logit.std(dim=-1, keepdim=True)
    return (logit - mean) / (1e-7 + stdv)


In [131]:
# CA-KLD Loss for Classification
def cakld_loss(student_logits, teacher_logits, beta_prob):
    # Forward KL (student || teacher)
    student_log_prob = F.log_softmax(student_logits, dim=1)
    teacher_prob = F.softmax(teacher_logits, dim=1)
    forward_kl = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

    # Reverse KL (teacher || student)
    teacher_log_prob = F.log_softmax(teacher_logits, dim=1)
    student_prob = F.softmax(student_logits, dim=1)
    reverse_kl = F.kl_div(teacher_log_prob, student_prob, reduction='batchmean')

    # Combined KL loss
    kl_loss = beta_prob * reverse_kl + (1 - beta_prob) * forward_kl
    return kl_loss


In [132]:
def evaluate(model, test_loader, device):
    model = model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [133]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [134]:
import torch
import time
def measure_inference_time(model, test_loader, num_runs=5):
    device = torch.device('cpu')
    model.eval()
    model.to(device)

    # Warm-up (one batch to avoid startup cost)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break

    total_time = 0
    total_images = 0

    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                start_time = time.time()
                _ = model(inputs)
                end_time = time.time()

                total_time += (end_time - start_time)
                total_images += batch_size

    avg_time_per_image = total_time / total_images
    return avg_time_per_image


In [135]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def calculate_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / (1024 * 1024)  # Size in MB
    os.remove(filename)
    return size

def compare_model_sizes(teacher, student, pruned_student):
    # Count parameters
    teacher_params = count_parameters(teacher)
    student_params = count_parameters(student)
    pruned_params = count_parameters(pruned_student)
    
    # Calculate disk size
    teacher_size = calculate_model_size(teacher, "teacher.pth")
    student_size = calculate_model_size(student, "student.pth")
    pruned_size = calculate_model_size(pruned_student, "pruned_student.pth")
    
    # Print comparison
    print("\n--- Model Size Comparison ---")
    print(f"Teacher Model: {teacher_params} parameters, {teacher_size:.2f} MB")
    print(f"Student Model (Before Pruning): {student_params} parameters, {student_size:.2f} MB")
    print(f"Student Model (After Pruning): {pruned_params} parameters, {pruned_size:.2f} MB")
    
    # Calculate compression ratio
    compression_ratio = student_size / pruned_size
    print(f"\nCompression Ratio: {compression_ratio:.2f}x")

In [136]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        print(epoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [137]:
def compute_gradient_importance(
    teacher, student, data_loader, device, temperature=4.0, alpha=0.5, beta_prob=0.5, accumulation_epochs=3
):
    importance_scores = {}

    # Initialize importance score storage for conv layer weights only
    for name, param in student.named_parameters():
        if 'weight' in name and len(param.shape) == 4:  # Conv weights only
            importance_scores[name] = torch.zeros_like(param.data, device=device)

    teacher.to(device).eval()
    student.to(device).train()

    # Add momentum for gradient accumulation smoothing
    momentum = 0.9  # Controls exponential moving average
    accumulated_batches = 0  # Track for bias correction

    for epoch in range(accumulation_epochs):
        print(f"Accumulation Epoch {epoch+1}/{accumulation_epochs}")
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            student.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature

            # Compute losses
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss

            # Modified backward propagation
            loss.backward()

            # Accumulate importance scores with parameter-gradient product
            accumulated_batches += 1
            for name, param in student.named_parameters():
                if name in importance_scores and param.grad is not None:
                    # Key modification: Use parameter-gradient product magnitude
                    grad_product = (param.data * param.grad).abs_()
                    
                    # Exponential moving average with bias correction
                    if accumulated_batches == 1:
                        importance_scores[name] = grad_product
                    else:
                        importance_scores[name] = momentum * importance_scores[name] + (1 - momentum) * grad_product

    # Apply bias correction for EMA
    for name in importance_scores:
        importance_scores[name] /= (1 - momentum**accumulated_batches)

    return importance_scores

In [138]:
def gradient_based_global_prune(model, importance_scores, prune_ratio=0.95):
    all_scores = torch.cat([score.flatten() for score in importance_scores.values()])
    threshold = torch.topk(all_scores, k=int(prune_ratio * all_scores.numel()), largest=False)[0][-1]

    for name, param in model.named_parameters():
        if name in importance_scores:
            mask = (importance_scores[name] > threshold).float()
            param.data.mul_(mask)

    return model


In [139]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [140]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# KD training with CA-KLD loss and mask-based momentum handling
def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50,
                    temperature=5.0, alpha=0.5, beta_prob=0.5, patience=5,
                    save_path="student_before_pruning.pth"):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks and zero momentum buffers
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:
            mask = (param != 0).float().to(device)
            masks[name] = mask
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    teacher = teacher.to(device).eval()
    student = student.to(device)

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)


            # CA-KLD loss
            kd_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)

            loss = alpha * kd_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()

            # Reapply masks and update momentum
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student) * 100.0  # Assuming this function is defined elsewhere

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Restore and save best model
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [141]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Training function with KD + CA-KLD and logits normalization
def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50, temperature=5.0, alpha=0.5,
                     beta_prob=0.5, patience=5, save_path="student_before_pruning.pth"):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    teacher = teacher.to(device)
    student = student.to(device)
    teacher.eval()  # Freeze teacher

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)

            # CA-KLD loss (normalized logits)
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ground_truth_loss = F.cross_entropy(student_logits, labels)

            # Combined loss
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation accuracy
        val_acc = evaluate(student, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Load best model state and save
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")

    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [142]:

student = train_kd_pruning(
    teacher, student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5,beta_prob=0.5, patience=5,save_path="student_before_pruning.pth"
)


Epoch 1/50 | Train Loss: 8.9872 | Train Acc: 55.35% | Val Acc: 61.43%
Epoch 2/50 | Train Loss: 4.2724 | Train Acc: 74.34% | Val Acc: 69.79%
Epoch 3/50 | Train Loss: 2.6246 | Train Acc: 82.81% | Val Acc: 75.91%
Epoch 4/50 | Train Loss: 1.6964 | Train Acc: 88.44% | Val Acc: 77.66%
Epoch 5/50 | Train Loss: 1.1719 | Train Acc: 91.85% | Val Acc: 81.27%
Epoch 6/50 | Train Loss: 0.8926 | Train Acc: 94.14% | Val Acc: 82.41%
Epoch 7/50 | Train Loss: 0.7277 | Train Acc: 95.22% | Val Acc: 82.70%
Epoch 8/50 | Train Loss: 0.6366 | Train Acc: 95.91% | Val Acc: 82.97%
Epoch 9/50 | Train Loss: 0.5762 | Train Acc: 96.11% | Val Acc: 83.16%
Epoch 10/50 | Train Loss: 0.5320 | Train Acc: 96.41% | Val Acc: 83.47%
Epoch 11/50 | Train Loss: 0.4964 | Train Acc: 96.52% | Val Acc: 83.72%
Epoch 12/50 | Train Loss: 0.4735 | Train Acc: 96.66% | Val Acc: 83.62%
Epoch 13/50 | Train Loss: 0.4522 | Train Acc: 96.73% | Val Acc: 83.54%
Epoch 14/50 | Train Loss: 0.4309 | Train Acc: 96.88% | Val Acc: 83.80%
Epoch 15/50 | T

In [144]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 80.89%
Student Model Test Accuracy Before Pruning: 82.77%


## 93% Sparsity

In [145]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [146]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 7s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [147]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 1.00%


In [148]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 70.24%
Epoch 1/200 | Train Loss: 1.8297 | Train Acc: 58.28%
Validation Loss: 1.0416 | Validation Acc: 70.24% | Sparsity: 93.74%

New best model saved with Val Accuracy: 74.30%
Epoch 2/200 | Train Loss: 0.6932 | Train Acc: 79.52%
Validation Loss: 0.8664 | Validation Acc: 74.30% | Sparsity: 93.74%

New best model saved with Val Accuracy: 75.65%
Epoch 3/200 | Train Loss: 0.5022 | Train Acc: 85.05%
Validation Loss: 0.8188 | Validation Acc: 75.65% | Sparsity: 93.74%

New best model saved with Val Accuracy: 76.22%
Epoch 4/200 | Train Loss: 0.3816 | Train Acc: 88.84%
Validation Loss: 0.7976 | Validation Acc: 76.22% | Sparsity: 93.74%

New best model saved with Val Accuracy: 76.60%
Epoch 5/200 | Train Loss: 0.2913 | Train Acc: 91.92%
Validation Loss: 0.8050 | Validation Acc: 76.60% | Sparsity: 93.74%

New best model saved with Val Accuracy: 76.66%
Epoch 6/200 | Train Loss: 0.2217 | Train Acc: 94.08%
Validation Loss: 0.8079 | Validation Acc: 76.66% | Spar

In [149]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 75.35%


In [150]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [151]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 8s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [152]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 7.6947 | Train Acc: 68.52% | Val Loss: 1.2570 | Val Acc: 69.73% | Sparsity: 93.74%
Epoch 2/50 | Train Loss: 3.9268 | Train Acc: 80.91% | Val Loss: 1.1996 | Val Acc: 72.09% | Sparsity: 93.74%
Epoch 3/50 | Train Loss: 2.7308 | Train Acc: 86.07% | Val Loss: 0.9457 | Val Acc: 75.52% | Sparsity: 93.74%
Epoch 4/50 | Train Loss: 2.0303 | Train Acc: 89.30% | Val Loss: 0.9088 | Val Acc: 77.41% | Sparsity: 93.74%
Epoch 5/50 | Train Loss: 1.5740 | Train Acc: 91.55% | Val Loss: 0.8357 | Val Acc: 78.34% | Sparsity: 93.74%
Epoch 6/50 | Train Loss: 1.3034 | Train Acc: 93.00% | Val Loss: 0.7681 | Val Acc: 79.74% | Sparsity: 93.74%
Epoch 7/50 | Train Loss: 1.1211 | Train Acc: 93.82% | Val Loss: 0.7619 | Val Acc: 79.94% | Sparsity: 93.74%
Epoch 8/50 | Train Loss: 1.0169 | Train Acc: 94.32% | Val Loss: 0.7269 | Val Acc: 80.64% | Sparsity: 93.74%
Epoch 9/50 | Train Loss: 0.9292 | Train Acc: 94.70% | Val Loss: 0.7386 | Val Acc: 80.26% | Sparsity: 93.74%
Epoch 10/50 | Train Loss: 0.

In [153]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 79.78%


In [154]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [155]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 5s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 2.3252 | Train Acc: 75.41% | Val Loss: 0.8594 | Val Acc: 76.06% | Sparsity: 93.74%
Epoch 2/50 | Train Loss: 1.0102 | Train Acc: 86.93% | Val Loss: 0.7885 | Val Acc: 78.24% | Sparsity: 93.74%
Epoch 3/50 | Train Loss: 0.7246 | Train Acc: 90.75% | Val Loss: 0.7452 | Val Acc: 78.97% | Sparsity: 93.74%
Epoch 4/50 | Train Loss: 0.5727 | Train Acc: 92.87% | Val Loss: 0.7167 | Val Acc: 79.53% | Sparsity: 93.74%
Epoch 5/50 | Train Loss: 0.4865 | Train Acc: 94.19% | Val Loss: 0.7155 | Val Acc: 80.35% | Sparsity: 93.74%
Epoch 6/50 | Train Loss: 0.4306 | Train Acc: 94.96% | Val Loss: 0.7092 | Val Acc: 80.02% | Sparsity: 93.74%
Epoch 7/50 | Train Loss: 0.3925 | Train Acc: 95.42% | Val Loss: 0.6973 | Val Acc: 80.73% | Sparsity: 93.74%
Epoch 8/50 | Train Loss: 0.3674 | Train Acc: 95.63% | Val Loss: 0.7033 | Val Acc: 80.72% | Sparsity: 93.74%
Epoch 9/50 | Train Loss: 0.3425 | Train Acc: 96.08% | Val Loss: 0.7000 | Val Acc: 80.46% | Sparsity: 93.74%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 79.54%


# 98% of Sparsity

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 7s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=7
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 9.81%
Epoch 1/200 | Train Loss: 4.3498 | Train Acc: 5.29%
Validation Loss: 3.9287 | Validation Acc: 9.81% | Sparsity: 98.22%

New best model saved with Val Accuracy: 21.33%
Epoch 2/200 | Train Loss: 3.5276 | Train Acc: 15.96%
Validation Loss: 3.1741 | Validation Acc: 21.33% | Sparsity: 98.22%

New best model saved with Val Accuracy: 42.70%
Epoch 3/200 | Train Loss: 2.6189 | Train Acc: 32.60%
Validation Loss: 2.1166 | Validation Acc: 42.70% | Sparsity: 98.22%

New best model saved with Val Accuracy: 53.83%
Epoch 4/200 | Train Loss: 1.8147 | Train Acc: 50.09%
Validation Loss: 1.6431 | Validation Acc: 53.83% | Sparsity: 98.22%

New best model saved with Val Accuracy: 58.58%
Epoch 5/200 | Train Loss: 1.4557 | Train Acc: 58.74%
Validation Loss: 1.4420 | Validation Acc: 58.58% | Sparsity: 98.22%

New best model saved with Val Accuracy: 61.69%
Epoch 6/200 | Train Loss: 1.2473 | Train Acc: 64.17%
Validation Loss: 1.3328 | Validation Acc: 61.69% | Sparsit

In [ ]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 66.14%


In [ ]:

# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f"Pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")


# Retrained with KD

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
student_accuracy = evaluate(student, test_loader, device)
print(f"Student Model Test Accuracy: {student_accuracy:.2f}%")

Student Model Test Accuracy: 82.77%


In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 2s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 17.0927 | Train Acc: 41.51% | Val Loss: 1.8627 | Val Acc: 53.75% | Sparsity: 98.22%
Epoch 2/50 | Train Loss: 9.1428 | Train Acc: 62.62% | Val Loss: 1.5847 | Val Acc: 59.67% | Sparsity: 98.22%
Epoch 3/50 | Train Loss: 7.2775 | Train Acc: 69.38% | Val Loss: 1.3506 | Val Acc: 64.69% | Sparsity: 98.22%
Epoch 4/50 | Train Loss: 6.1327 | Train Acc: 73.05% | Val Loss: 1.2992 | Val Acc: 66.63% | Sparsity: 98.22%
Epoch 5/50 | Train Loss: 5.3373 | Train Acc: 75.99% | Val Loss: 1.2075 | Val Acc: 67.96% | Sparsity: 98.22%
Epoch 6/50 | Train Loss: 4.7834 | Train Acc: 78.03% | Val Loss: 1.2672 | Val Acc: 67.49% | Sparsity: 98.22%
Epoch 7/50 | Train Loss: 4.3619 | Train Acc: 79.83% | Val Loss: 1.1970 | Val Acc: 68.54% | Sparsity: 98.22%
Epoch 8/50 | Train Loss: 3.9138 | Train Acc: 81.86% | Val Loss: 1.1688 | Val Acc: 69.82% | Sparsity: 98.22%
Epoch 9/50 | Train Loss: 3.5504 | Train Acc: 83.03% | Val Loss: 1.1340 | Val Acc: 70.51% | Sparsity: 98.22%
Epoch 10/50 | Train Loss: 3

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 72.79%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 6s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 6.8956 | Train Acc: 40.48% | Val Loss: 1.5925 | Val Acc: 57.35% | Sparsity: 98.22%
Epoch 2/50 | Train Loss: 3.3404 | Train Acc: 64.96% | Val Loss: 1.2948 | Val Acc: 64.17% | Sparsity: 98.22%
Epoch 3/50 | Train Loss: 2.5846 | Train Acc: 71.34% | Val Loss: 1.1690 | Val Acc: 67.97% | Sparsity: 98.22%
Epoch 4/50 | Train Loss: 2.1853 | Train Acc: 75.09% | Val Loss: 1.1041 | Val Acc: 69.52% | Sparsity: 98.22%
Epoch 5/50 | Train Loss: 1.9072 | Train Acc: 78.19% | Val Loss: 1.1432 | Val Acc: 68.39% | Sparsity: 98.22%
Epoch 6/50 | Train Loss: 1.7025 | Train Acc: 80.27% | Val Loss: 1.0923 | Val Acc: 69.96% | Sparsity: 98.22%
Epoch 7/50 | Train Loss: 1.5268 | Train Acc: 82.30% | Val Loss: 1.0550 | Val Acc: 70.74% | Sparsity: 98.22%
Epoch 8/50 | Train Loss: 1.4092 | Train Acc: 83.67% | Val Loss: 1.0414 | Val Acc: 71.15% | Sparsity: 98.22%
Epoch 9/50 | Train Loss: 1.2815 | Train Acc: 85.17% | Val Loss: 1.0263 | Val Acc: 71.36% | Sparsity: 98.22%
Epoch 10/50 | Train Loss: 1.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 72.38%


## 90% Sparsity

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.91)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 5s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 75.54%
Epoch 1/100 | Train Loss: 1.1331 | Train Acc: 74.40%
Validation Loss: 0.8176 | Validation Acc: 75.54% | Sparsity: 90.75%

New best model saved with Val Accuracy: 77.57%
Epoch 2/100 | Train Loss: 0.4314 | Train Acc: 87.36%
Validation Loss: 0.7451 | Validation Acc: 77.57% | Sparsity: 90.75%

New best model saved with Val Accuracy: 78.02%
Epoch 3/100 | Train Loss: 0.2947 | Train Acc: 91.50%
Validation Loss: 0.7213 | Validation Acc: 78.02% | Sparsity: 90.75%

New best model saved with Val Accuracy: 78.66%
Epoch 4/100 | Train Loss: 0.2043 | Train Acc: 94.47%
Validation Loss: 0.7312 | Validation Acc: 78.66% | Sparsity: 90.75%

Epoch 5/100 | Train Loss: 0.1413 | Train Acc: 96.50%
Validation Loss: 0.7575 | Validation Acc: 78.62% | Sparsity: 90.75%

Epoch 6/100 | Train Loss: 0.0958 | Train Acc: 98.10%
Validation Loss: 0.7824 | Validation Acc: 78.45% | Sparsity: 90.75%

Epoch 7/100 | Train Loss: 0.0646 | Train Acc: 98.96%
Validation Loss: 0.8168 | V

In [ ]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 77.89%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.91)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 3s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 5.3654 | Train Acc: 76.50% | Val Loss: 1.1000 | Val Acc: 71.91% | Sparsity: 90.75%
Epoch 2/50 | Train Loss: 2.6587 | Train Acc: 86.45% | Val Loss: 0.9261 | Val Acc: 76.34% | Sparsity: 90.75%
Epoch 3/50 | Train Loss: 1.7471 | Train Acc: 90.65% | Val Loss: 0.8116 | Val Acc: 79.22% | Sparsity: 90.75%
Epoch 4/50 | Train Loss: 1.3214 | Train Acc: 92.89% | Val Loss: 0.7560 | Val Acc: 80.03% | Sparsity: 90.75%
Epoch 5/50 | Train Loss: 1.0560 | Train Acc: 94.11% | Val Loss: 0.7010 | Val Acc: 81.12% | Sparsity: 90.75%
Epoch 6/50 | Train Loss: 0.9199 | Train Acc: 94.76% | Val Loss: 0.6706 | Val Acc: 81.66% | Sparsity: 90.75%
Epoch 7/50 | Train Loss: 0.8292 | Train Acc: 95.08% | Val Loss: 0.6528 | Val Acc: 82.02% | Sparsity: 90.75%
Epoch 8/50 | Train Loss: 0.7669 | Train Acc: 95.18% | Val Loss: 0.6556 | Val Acc: 82.05% | Sparsity: 90.75%
Epoch 9/50 | Train Loss: 0.7148 | Train Acc: 95.48% | Val Loss: 0.6431 | Val Acc: 82.11% | Sparsity: 90.75%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 81.44%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.91)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.5190 | Train Acc: 83.26% | Val Loss: 0.7699 | Val Acc: 78.36% | Sparsity: 90.75%
Epoch 2/50 | Train Loss: 0.6432 | Train Acc: 91.43% | Val Loss: 0.7063 | Val Acc: 79.79% | Sparsity: 90.75%
Epoch 3/50 | Train Loss: 0.4737 | Train Acc: 93.89% | Val Loss: 0.6700 | Val Acc: 80.91% | Sparsity: 90.75%
Epoch 4/50 | Train Loss: 0.3940 | Train Acc: 95.08% | Val Loss: 0.6481 | Val Acc: 81.64% | Sparsity: 90.75%
Epoch 5/50 | Train Loss: 0.3504 | Train Acc: 95.68% | Val Loss: 0.6513 | Val Acc: 81.81% | Sparsity: 90.75%
Epoch 6/50 | Train Loss: 0.3183 | Train Acc: 95.97% | Val Loss: 0.6422 | Val Acc: 82.08% | Sparsity: 90.75%
Epoch 7/50 | Train Loss: 0.2998 | Train Acc: 96.29% | Val Loss: 0.6433 | Val Acc: 81.94% | Sparsity: 90.75%
Epoch 8/50 | Train Loss: 0.2834 | Train Acc: 96.48% | Val Loss: 0.6470 | Val Acc: 81.78% | Sparsity: 90.75%
Epoch 9/50 | Train Loss: 0.2721 | Train Acc: 96.57% | Val Loss: 0.6489 | Val Acc: 81.63% | Sparsity: 90.75%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 81.09%


# 80% Sparsity

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.81)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 3s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 1.14%


In [ ]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 81.55%
Epoch 1/100 | Train Loss: 0.3405 | Train Acc: 92.36%
Validation Loss: 0.6221 | Validation Acc: 81.55% | Sparsity: 80.77%

New best model saved with Val Accuracy: 81.92%
Epoch 2/100 | Train Loss: 0.1411 | Train Acc: 96.31%
Validation Loss: 0.6223 | Validation Acc: 81.92% | Sparsity: 80.77%

New best model saved with Val Accuracy: 82.24%
Epoch 3/100 | Train Loss: 0.0777 | Train Acc: 98.30%
Validation Loss: 0.6408 | Validation Acc: 82.24% | Sparsity: 80.77%

New best model saved with Val Accuracy: 82.27%
Epoch 4/100 | Train Loss: 0.0411 | Train Acc: 99.38%
Validation Loss: 0.6640 | Validation Acc: 82.27% | Sparsity: 80.77%

New best model saved with Val Accuracy: 82.38%
Epoch 5/100 | Train Loss: 0.0223 | Train Acc: 99.81%
Validation Loss: 0.6837 | Validation Acc: 82.38% | Sparsity: 80.77%

Epoch 6/100 | Train Loss: 0.0144 | Train Acc: 99.92%
Validation Loss: 0.7026 | Validation Acc: 82.16% | Sparsity: 80.77%

Epoch 7/100 | Train Loss: 0.0101 

In [ ]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 81.09%


In [ ]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.81)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.7182 | Train Acc: 91.61% | Val Loss: 0.7152 | Val Acc: 80.61% | Sparsity: 80.77%
Epoch 2/50 | Train Loss: 1.0003 | Train Acc: 94.42% | Val Loss: 0.6445 | Val Acc: 81.98% | Sparsity: 80.77%
Epoch 3/50 | Train Loss: 0.7656 | Train Acc: 95.24% | Val Loss: 0.6271 | Val Acc: 82.60% | Sparsity: 80.77%
Epoch 4/50 | Train Loss: 0.6587 | Train Acc: 95.48% | Val Loss: 0.5900 | Val Acc: 83.28% | Sparsity: 80.77%
Epoch 5/50 | Train Loss: 0.6001 | Train Acc: 95.95% | Val Loss: 0.5946 | Val Acc: 83.38% | Sparsity: 80.77%
Epoch 6/50 | Train Loss: 0.5556 | Train Acc: 95.78% | Val Loss: 0.5800 | Val Acc: 83.55% | Sparsity: 80.77%
Epoch 7/50 | Train Loss: 0.5190 | Train Acc: 95.97% | Val Loss: 0.5825 | Val Acc: 83.68% | Sparsity: 80.77%
Epoch 8/50 | Train Loss: 0.5023 | Train Acc: 95.98% | Val Loss: 0.5856 | Val Acc: 83.31% | Sparsity: 80.77%
Epoch 9/50 | Train Loss: 0.4875 | Train Acc: 96.07% | Val Loss: 0.5814 | Val Acc: 83.72% | Sparsity: 80.77%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 81.91%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.81)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 6s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.5290 | Train Acc: 93.96% | Val Loss: 0.6048 | Val Acc: 82.81% | Sparsity: 80.77%
Epoch 2/50 | Train Loss: 0.2883 | Train Acc: 96.08% | Val Loss: 0.5885 | Val Acc: 82.87% | Sparsity: 80.77%
Epoch 3/50 | Train Loss: 0.2448 | Train Acc: 96.66% | Val Loss: 0.5770 | Val Acc: 83.28% | Sparsity: 80.77%
Epoch 4/50 | Train Loss: 0.2234 | Train Acc: 96.84% | Val Loss: 0.5761 | Val Acc: 83.48% | Sparsity: 80.77%
Epoch 5/50 | Train Loss: 0.2075 | Train Acc: 97.10% | Val Loss: 0.5773 | Val Acc: 83.37% | Sparsity: 80.77%
Epoch 6/50 | Train Loss: 0.1992 | Train Acc: 97.07% | Val Loss: 0.5767 | Val Acc: 83.16% | Sparsity: 80.77%
Epoch 7/50 | Train Loss: 0.1920 | Train Acc: 97.20% | Val Loss: 0.5763 | Val Acc: 83.42% | Sparsity: 80.77%
Epoch 8/50 | Train Loss: 0.1870 | Train Acc: 97.19% | Val Loss: 0.5773 | Val Acc: 83.50% | Sparsity: 80.77%
Epoch 9/50 | Train Loss: 0.1818 | Train Acc: 97.20% | Val Loss: 0.5756 | Val Acc: 83.34% | Sparsity: 80.77%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 82.21%


# 75% of Sparsity

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 3.73%


In [ ]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_75.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 82.74%
Epoch 1/100 | Train Loss: 0.2018 | Train Acc: 95.19%
Validation Loss: 0.5919 | Validation Acc: 82.74% | Sparsity: 74.85%

New best model saved with Val Accuracy: 82.89%
Epoch 2/100 | Train Loss: 0.0895 | Train Acc: 97.83%
Validation Loss: 0.6123 | Validation Acc: 82.89% | Sparsity: 74.85%

Epoch 3/100 | Train Loss: 0.0444 | Train Acc: 99.23%
Validation Loss: 0.6392 | Validation Acc: 82.70% | Sparsity: 74.85%

New best model saved with Val Accuracy: 82.99%
Epoch 4/100 | Train Loss: 0.0217 | Train Acc: 99.81%
Validation Loss: 0.6550 | Validation Acc: 82.99% | Sparsity: 74.85%

New best model saved with Val Accuracy: 83.20%
Epoch 5/100 | Train Loss: 0.0137 | Train Acc: 99.92%
Validation Loss: 0.6587 | Validation Acc: 83.20% | Sparsity: 74.85%

Epoch 6/100 | Train Loss: 0.0092 | Train Acc: 99.95%
Validation Loss: 0.6694 | Validation Acc: 83.09% | Sparsity: 74.85%

Epoch 7/100 | Train Loss: 0.0069 | Train Acc: 99.97%
Validation Loss: 0.6815 | V

In [ ]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 81.70%


In [ ]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

# Retrain with KD

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 3s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.0007 | Train Acc: 94.87% | Val Loss: 0.6224 | Val Acc: 82.23% | Sparsity: 74.85%
Epoch 2/50 | Train Loss: 0.6937 | Train Acc: 95.58% | Val Loss: 0.6009 | Val Acc: 83.03% | Sparsity: 74.85%
Epoch 3/50 | Train Loss: 0.5904 | Train Acc: 95.77% | Val Loss: 0.5831 | Val Acc: 83.58% | Sparsity: 74.85%
Epoch 4/50 | Train Loss: 0.5346 | Train Acc: 95.99% | Val Loss: 0.5719 | Val Acc: 83.46% | Sparsity: 74.85%
Epoch 5/50 | Train Loss: 0.5063 | Train Acc: 95.94% | Val Loss: 0.5622 | Val Acc: 84.02% | Sparsity: 74.85%
Epoch 6/50 | Train Loss: 0.4791 | Train Acc: 96.03% | Val Loss: 0.5722 | Val Acc: 83.67% | Sparsity: 74.85%
Epoch 7/50 | Train Loss: 0.4567 | Train Acc: 96.14% | Val Loss: 0.5636 | Val Acc: 83.75% | Sparsity: 74.85%
Epoch 8/50 | Train Loss: 0.4442 | Train Acc: 96.12% | Val Loss: 0.5624 | Val Acc: 83.72% | Sparsity: 74.85%
Epoch 9/50 | Train Loss: 0.4273 | Train Acc: 96.19% | Val Loss: 0.5700 | Val Acc: 83.37% | Sparsity: 74.85%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 82.25%


In [ ]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.3480 | Train Acc: 95.70% | Val Loss: 0.5682 | Val Acc: 83.23% | Sparsity: 74.85%
Epoch 2/50 | Train Loss: 0.2246 | Train Acc: 96.70% | Val Loss: 0.5605 | Val Acc: 83.62% | Sparsity: 74.85%
Epoch 3/50 | Train Loss: 0.1993 | Train Acc: 97.04% | Val Loss: 0.5564 | Val Acc: 83.54% | Sparsity: 74.85%
Epoch 4/50 | Train Loss: 0.1857 | Train Acc: 97.17% | Val Loss: 0.5544 | Val Acc: 83.63% | Sparsity: 74.85%
Epoch 5/50 | Train Loss: 0.1767 | Train Acc: 97.27% | Val Loss: 0.5539 | Val Acc: 83.93% | Sparsity: 74.85%
Epoch 6/50 | Train Loss: 0.1706 | Train Acc: 97.27% | Val Loss: 0.5525 | Val Acc: 84.20% | Sparsity: 74.85%
Epoch 7/50 | Train Loss: 0.1656 | Train Acc: 97.33% | Val Loss: 0.5545 | Val Acc: 83.89% | Sparsity: 74.85%
Epoch 8/50 | Train Loss: 0.1607 | Train Acc: 97.32% | Val Loss: 0.5532 | Val Acc: 83.69% | Sparsity: 74.85%
Epoch 9/50 | Train Loss: 0.1583 | Train Acc: 97.41% | Val Loss: 0.5531 | Val Acc: 84.11% | Sparsity: 74.85%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 82.41%


## 50% Sparsity

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.51)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 5s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 70.74%


In [ ]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model_50%.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 84.56%
Epoch 1/200 | Train Loss: 0.0912 | Train Acc: 97.17%
Validation Loss: 0.5540 | Validation Acc: 84.56% | Sparsity: 50.86%

Epoch 2/200 | Train Loss: 0.0382 | Train Acc: 99.04%
Validation Loss: 0.5882 | Validation Acc: 84.34% | Sparsity: 50.86%

Epoch 3/200 | Train Loss: 0.0168 | Train Acc: 99.79%
Validation Loss: 0.6084 | Validation Acc: 84.27% | Sparsity: 50.86%

New best model saved with Val Accuracy: 84.64%
Epoch 4/200 | Train Loss: 0.0098 | Train Acc: 99.92%
Validation Loss: 0.6134 | Validation Acc: 84.64% | Sparsity: 50.86%

Epoch 5/200 | Train Loss: 0.0065 | Train Acc: 99.96%
Validation Loss: 0.6214 | Validation Acc: 84.56% | Sparsity: 50.86%

Epoch 6/200 | Train Loss: 0.0050 | Train Acc: 99.97%
Validation Loss: 0.6364 | Validation Acc: 84.45% | Sparsity: 50.86%

Epoch 7/200 | Train Loss: 0.0040 | Train Acc: 99.98%
Validation Loss: 0.6402 | Validation Acc: 84.61% | Sparsity: 50.86%

Epoch 8/200 | Train Loss: 0.0035 | Train Acc: 99.98%

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 83.18%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.51)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.4399 | Train Acc: 97.09% | Val Loss: 0.5499 | Val Acc: 84.19% | Sparsity: 50.86%
Epoch 2/50 | Train Loss: 0.4146 | Train Acc: 96.54% | Val Loss: 0.5534 | Val Acc: 84.66% | Sparsity: 50.86%
Epoch 3/50 | Train Loss: 0.3946 | Train Acc: 96.41% | Val Loss: 0.5542 | Val Acc: 83.99% | Sparsity: 50.86%
Epoch 4/50 | Train Loss: 0.3818 | Train Acc: 96.38% | Val Loss: 0.5532 | Val Acc: 84.46% | Sparsity: 50.86%
Epoch 5/50 | Train Loss: 0.3733 | Train Acc: 96.37% | Val Loss: 0.5522 | Val Acc: 84.11% | Sparsity: 50.86%
Epoch 6/50 | Train Loss: 0.3678 | Train Acc: 96.42% | Val Loss: 0.5505 | Val Acc: 84.48% | Sparsity: 50.86%
Epoch 7/50 | Train Loss: 0.3570 | Train Acc: 96.34% | Val Loss: 0.5513 | Val Acc: 84.36% | Sparsity: 50.86%
Early stopping triggered at epoch 7. No improvement for 5 epochs.
Student model saved before pruning at: pruned_student_retrain_KD_90%.pth
Total Training Time: 7m 59s
Retraining completed in 7.98 minutes (478.80 seconds)


In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 82.83%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.51)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 3s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.1621 | Train Acc: 97.33% | Val Loss: 0.5357 | Val Acc: 84.59% | Sparsity: 50.86%
Epoch 2/50 | Train Loss: 0.1420 | Train Acc: 97.46% | Val Loss: 0.5385 | Val Acc: 84.65% | Sparsity: 50.86%
Epoch 3/50 | Train Loss: 0.1350 | Train Acc: 97.65% | Val Loss: 0.5383 | Val Acc: 84.52% | Sparsity: 50.86%
Epoch 4/50 | Train Loss: 0.1310 | Train Acc: 97.62% | Val Loss: 0.5379 | Val Acc: 84.61% | Sparsity: 50.86%
Epoch 5/50 | Train Loss: 0.1287 | Train Acc: 97.69% | Val Loss: 0.5361 | Val Acc: 84.73% | Sparsity: 50.86%
Epoch 6/50 | Train Loss: 0.1273 | Train Acc: 97.64% | Val Loss: 0.5403 | Val Acc: 84.82% | Sparsity: 50.86%
Epoch 7/50 | Train Loss: 0.1258 | Train Acc: 97.72% | Val Loss: 0.5384 | Val Acc: 84.62% | Sparsity: 50.86%
Epoch 8/50 | Train Loss: 0.1249 | Train Acc: 97.75% | Val Loss: 0.5423 | Val Acc: 84.56% | Sparsity: 50.86%
Epoch 9/50 | Train Loss: 0.1226 | Train Acc: 97.73% | Val Loss: 0.5415 | Val Acc: 84.76% | Sparsity: 50.86%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 83.12%
